In [0]:
!pip install pymongo
!pip install confluent_kafka


# COMMAND ----------

from pymongo.mongo_client import MongoClient
from confluent_kafka import Consumer, KafkaException
import json
import time


KAFKA_BOOTSTRAP_SERVER = 'pkc-w77k7w.centralus.azure.confluent.cloud:9092'
KAFKA_API_KEY = '36QWHWHZ6CMQYPKH'
KAFKA_API_SECRET = 'Your Key'
KAFKA_TOPIC = "transaction_predict"
KAFKA_GROUP_ID = "fraud-detection-group"
MONGO_URI = "mongodb+srv://Devendra:<Your Password>@devendra.e3kfvth.mongodb.net/?retryWrites=true&w=majority&appName=Devendra"
MONGO_DB = "transaction_predict"
MONGO_COLLECTION = "transaction"

client = MongoClient(MONGO_URI)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]
collection_prediction = db['prediction_data']


conf = {
    'bootstrap.servers': KAFKA_BOOTSTRAP_SERVER,
    'security.protocol': 'SASL_SSL',  # Fixed typo here
    'sasl.mechanism': 'PLAIN',
    'sasl.username': KAFKA_API_KEY,
    'sasl.password': KAFKA_API_SECRET, 
    'group.id': KAFKA_GROUP_ID,
    'auto.offset.reset': 'earliest'
}

consumer = Consumer(conf)
consumer.subscribe([KAFKA_TOPIC])


You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-bc7c53f6-bf51-4a79-a612-ca68bf1fe1ae/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-bc7c53f6-bf51-4a79-a612-ca68bf1fe1ae/bin/python -m pip install --upgrade pip' command.


In [0]:
from pyspark.ml.regression import RandomForestRegressionModel

model = RandomForestRegressionModel.load(
    "/Workspace/Users/rahulrock2494@gmail.com/Transaction Prediction"
)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
import pandas as pd
spark = SparkSession.builder.appName("transaction_prediction").getOrCreate()


In [0]:
try:
    while True:
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            print(f"Kafka error: {msg.error()}")
            continue

        data = msg.value().decode('utf-8')
        json_data = json.loads(data)
        d = pd.DataFrame([json_data])
        spark_df = spark.createDataFrame(d)

        assembler = VectorAssembler(inputCols=['amount', 'location_id'], outputCol="features")
        assembled = assembler.transform(spark_df).select("features", "amount")

        predicted = model.transform(assembled).select("amount", "prediction").toPandas().to_dict(orient="records")
        print(predicted)
        print(predicted[0])
        collection_prediction.insert_one(predicted[0]) 
        
        print(json_data)
        collection.insert_one(json_data)

except KeyboardInterrupt:
    print("Stopping consumer...")

finally:
    consumer.close()

[{'amount': 4601.7, 'prediction': 4290.356901504532}]
{'amount': 4601.7, 'prediction': 4290.356901504532}
{'transaction_id': 'd98b6e5e-c883-44b3-a4a2-043f2e680b08', 'timestamp': 1755279266, 'user_id': 15909, 'amount': 4601.7, 'transaction_type': 'transfer', 'location': 'Lake Lindsey', 'location_id': 83594, 'merchant': 'Andersen-Castaneda', 'card_number': '4570589909760229'}
[{'amount': 3840.36, 'prediction': 4143.25494666831}]
{'amount': 3840.36, 'prediction': 4143.25494666831}
{'transaction_id': '68201c1c-0964-44f9-91ae-5cc2b95a2fff', 'timestamp': 1755279271, 'user_id': 54877, 'amount': 3840.36, 'transaction_type': 'withdrawal', 'location': 'New Shelby', 'location_id': 36814, 'merchant': 'Brown, Cooper and Harmon', 'card_number': '4980249824781396'}
[{'amount': 3034.59, 'prediction': 3118.593915834447}]
{'amount': 3034.59, 'prediction': 3118.593915834447}
{'transaction_id': '52d64a9f-8d82-4c8e-96c3-4993d9082969', 'timestamp': 1755279273, 'user_id': 11107, 'amount': 3034.59, 'transacti